In [25]:
import ctypes
from ctypes import c_void_p, c_int, POINTER, c_float
import numpy as np
import time
import random
import os
import platform

# Build command (rtx 4090), windows
# cd C:\Users\luka\source\repos\Artifical-Neural-Networks-From-Scratch\cuda\main
# nvcc -shared math_ops.cu nn_activations.cu nn_layers.cu -o nn.dll -arch=sm_89 -Xcompiler "/MD"

# Linux: 
# nvcc -shared ./cuda/main/math_ops.cu ./cuda/main/nn_layers.cu ./cuda/main/nn_activations.cu -o ./cuda/main/libnn.so -arch=sm_86 -Xcompiler -fPIC


In [26]:

class CudaNN:
    def __init__(self, dll_path="./main/nn.dll"):
        lib_path = None
        if lib_path is None:
            if platform.system() == "Windows":
                lib_path = "./main/nn.dll"       # Windows path
            else:
                lib_path = "./main/libnn.so" # Linux path
                
        # Jupyter runs from the directory the .ipynb file is in. 
        # Make sure this relative path is correct!
        if not os.path.exists(lib_path):
            raise FileNotFoundError(f"Library not found at {lib_path}. Check your notebook's current working directory: {os.getcwd()}")
        self.lib = ctypes.CDLL(dll_path)
        
        # Args and Outputs initialization
        
        self.lib.mat_add_cuda.argtypes = [
            POINTER(c_float), POINTER(c_float), POINTER(c_float),
            c_int, c_int
        ]
        self.lib.mat_add_cuda.restype = None


        self.lib.dot_product_cuda.argtypes = [
            POINTER(c_float), POINTER(c_float), POINTER(c_float), c_int
        ]
        self.lib.dot_product_cuda.restype = None

        self.lib.argmax_cuda.argtypes = [
            POINTER(c_float), POINTER(c_int), c_int
        ]
        self.lib.argmax_cuda.restype = None


        self.lib.relu_cuda.argtypes = [
            POINTER(c_float), c_int
        ]
        self.lib.relu_cuda.restype = None

        self.lib.relu_cuda_100.argtypes = [
            POINTER(c_float), c_int
        ]
        self.lib.relu_cuda_100.restype = None

        self.lib.create_dense_layer.argtypes = [c_int, c_int, POINTER(c_float), POINTER(c_float)]
        self.lib.create_dense_layer.restype = c_void_p # Returns a memory address

        self.lib.forward_dense_layer.argtypes = [c_void_p, c_void_p] # Takes layer pointer and device input pointer
        self.lib.forward_dense_layer.restype = c_void_p # Returns device output pointer

        self.lib.destroy_dense_layer.argtypes = [c_void_p]
        self.lib.destroy_dense_layer.restype = None

        self.lib.forward_dense_layer_host.argtypes = [c_void_p, POINTER(c_float), POINTER(c_float)]
        self.lib.forward_dense_layer_host.restype = None

        self.lib.copy_array_to_device.argtypes = [POINTER(c_float),c_int]
        self.lib.copy_array_to_device.restype = c_void_p

        self.lib.copy_device_array_to_host.argtypes = [c_void_p,c_int]
        self.lib.copy_device_array_to_host.restype = c_void_p

    def relu(self, arr):
        n = arr.size
        self.lib.relu_cuda(
            arr.ctypes.data_as(POINTER(c_float)),
            n
        )
        return arr

    def relu_100(self, arr):
        n = arr.size
        self.lib.relu_cuda_100(
            arr.ctypes.data_as(POINTER(c_float)),
            n
        )
        return arr

    def mat_add(self, A, B):
        rows, cols = A.shape
        C = np.zeros_like(A, dtype=np.float32)
        
        self.lib.mat_add_cuda(
            A.ctypes.data_as(POINTER(c_float)),
            B.ctypes.data_as(POINTER(c_float)),
            C.ctypes.data_as(POINTER(c_float)),
            rows, cols
        )
        return C

    def argmax(self, arr):
        n = arr.size
        result = c_int()
        
        self.lib.argmax_cuda(
            arr.ctypes.data_as(POINTER(c_float)),
            ctypes.byref(result),
            n
        )
        return result.value

    def dot_product(self, a, b):
        n = a.size
        res = c_float()
        
        self.lib.dot_product_cuda(
            a.ctypes.data_as(POINTER(c_float)),
            b.ctypes.data_as(POINTER(c_float)),
            ctypes.byref(res),
            n
        )
        return res.value
    
    
    class DenseLayer:
        def __init__(self, lib, weights, bias):
            self.lib = lib
            self.n_outputs, self.n_inputs = weights.shape 
        
            self.obj_ptr = self.lib.create_dense_layer(
                self.n_inputs, self.n_outputs, 
                weights.ctypes.data_as(POINTER(c_float)), 
                bias.ctypes.data_as(POINTER(c_float))
            )
            
        def forward(self, d_inputs):
            d_ptr = self.lib.forward_dense_layer(
                self.obj_ptr,
                d_inputs
            )
            return d_ptr
            
        def __del__(self): # prevents memory leak
            try:
                if self.obj_ptr:
                    self.lib.destroy_dense_layer(self.obj_ptr)
            except Exception as e:
                pass
    
    def copy_array_to_device(self,arr):
        return self.lib.copy_array_to_device(arr.ctypes.data_as(POINTER(c_float)),arr.size)

    def copy_device_array_to_host(self, d_arr, layer_obj: DenseLayer):
        h_ptr = self.lib.copy_device_array_to_host(d_arr, layer_obj.n_outputs)
        c_float_ptr = ctypes.cast(h_ptr, POINTER(c_float))
        
        host_array = np.ctypeslib.as_array(c_float_ptr, shape=(layer_obj.n_outputs,)).copy()
        
        return host_array


In [27]:
cuda_nn = CudaNN("./main/libnn.so")

In [28]:
# --- Test 1: Matrix Addition ---
rows, cols = 5120, 5120
A = np.random.rand(rows, cols).astype(np.float32)
B = np.random.rand(rows, cols).astype(np.float32)

start = time.time()
C = cuda_nn.mat_add(A, B) 
print(f"CUDA Add time: {time.time() - start:.4f}s")

CUDA Add time: 0.0722s


In [29]:
# --- Test 2: Dot Product ---
n = 10240
vec_a = np.ones(n, dtype=np.float32)
vec_b = np.ones(n, dtype=np.float32) * 3.0

dot_res = cuda_nn.dot_product(vec_a, vec_b)
print(f"Dot product: {dot_res}")

Dot product: 30720.0


In [30]:
# --- Test 4: Argmax ---
# Simulating a neural network's probability output for 5 classes
probs = np.array([0.05, 0.15, 0.60, 0.10, 0.10], dtype=np.float32)

cuda_idx = cuda_nn.argmax(probs)
numpy_idx = np.argmax(probs)

print(f"CUDA Argmax index:  {cuda_idx} (Value: {probs[cuda_idx]:.2f})")
print(f"Numpy Argmax index: {numpy_idx} (Value: {probs[numpy_idx]:.2f})")

if cuda_idx == numpy_idx:
    print("Success: CUDA matches NumPy")
else:
    print("Error: Results do not match")

CUDA Argmax index:  2 (Value: 0.60)
Numpy Argmax index: 2 (Value: 0.60)
Success: CUDA matches NumPy


In [31]:
# --- Test 5: ReLU Activation ---
# Array with a mix of positive and negative numbers

# 1,000,000 elements
N = 10_000_000
test_data = np.random.uniform(-10, 10, N).astype(np.float32)

# --- 1. Raw Python ---
python_list = test_data.tolist()
start = time.time()
python_res = [x if x > 0 else 0.0 for x in python_list]
python_time = time.time() - start

# --- 2. NumPy ---
start = time.time()
numpy_res = np.maximum(0, test_data)
numpy_time = time.time() - start

# --- 3. NumPy x100 ---
start = time.time()
for i in range(100): # This is to attempt to combat the overhead of CUDA when benchmarking
    numpy_res = np.maximum(0, test_data)
numpy_time_100 = time.time() - start

# --- 4. CUDA Kernel ---
cuda_data = test_data.copy()
start = time.time()
cuda_nn.relu(cuda_data)
cuda_time = time.time() - start

# --- 5. CUDA Kernel x100 ---
cuda_data_100 = test_data.copy()
start = time.time()
cuda_nn.relu_100(cuda_data_100)
cuda_time_100 = time.time() - start

# --- Results ---
print(f"Results for {N:,} elements:")
print(f"{'Method':<15} | {'Time (s)':<12} | {'Speedup vs Python':<18}")
print("-" * 50)
print(f"{'Raw Python':<15} | {python_time:<12.6f} | {'1.0x':<18}")
print(f"{'NumPy':<15} | {numpy_time:<12.6f} | {python_time/numpy_time:.2f}x")
print(f"{'NumPy (x100)':<15} | {numpy_time_100:<12.6f} | {python_time/numpy_time_100:.2f}x")
print(f"{'CUDA':<15} | {cuda_time:<12.6f} | {python_time/cuda_time:.2f}x")
print(f"{'CUDA (x100)':<15} | {cuda_time_100:<12.6f} | {python_time/cuda_time_100:.2f}x")

# Verification
if np.allclose(cuda_data, numpy_res):
    print("\n✅ Verification Successful: CUDA matches NumPy")

Results for 10,000,000 elements:
Method          | Time (s)     | Speedup vs Python 
--------------------------------------------------
Raw Python      | 0.420290     | 1.0x              
NumPy           | 0.009469     | 44.39x
NumPy (x100)    | 1.021148     | 0.41x
CUDA            | 0.017319     | 24.27x
CUDA (x100)     | 0.023290     | 18.05x

✅ Verification Successful: CUDA matches NumPy


In [32]:
# Dense layer OOP
input_vec = np.array([1.0, 2.0, 3.0, 4.0], dtype=np.float32)
weights = np.array([
    [0.1, 0.2, 0.3, 0.4], 
    [0.5, 0.6, 0.7, 0.8], 
    [0.9, 1.0, 1.1, 1.2]
], dtype=np.float32)
bias = np.array([0.1, 0.2, 0.3], dtype=np.float32)

d_inputs = cuda_nn.copy_array_to_device(input_vec)

layer1 = cuda_nn.DenseLayer(cuda_nn.lib,weights,bias)
d_output_ptr = layer1.forward(d_inputs)
host_output = cuda_nn.copy_device_array_to_host(d_output_ptr,layer1)
print(host_output)

[ 3.1  7.2 11.3]
